In [1]:
import pandas as pd
import warnings
import numpy as np
import seaborn as sns

# 2. 처리 작업

In [2]:
train =pd.read_csv('FIFA_train.csv')
test =pd.read_csv('FIFA_test.csv')
sub = pd.read_csv('submission.csv')

## 2.1. train, test 대응

In [3]:
train_test_dataset = [train, test]

In [4]:
for data in train_test_dataset:
    continent_mapping = {'europe': 0 ,'south america' : 1, 'asia' : 2, 'africa' : 3, 'oceania': 4}
    data['continent'] = data['continent'].map(continent_mapping)
    
    position_mapping = {'MF':0 , 'DF' : 1, 'ST' : 2, 'GK' : 3}
    data['position'] = data['position'].map(position_mapping)

    prefer_foot_mapping = {'right' :0, 'left': 1}
    data['prefer_foot'] = data['prefer_foot'].map(prefer_foot_mapping)
    
    def func(string):
        string=string[-4:]
        return int(string)

    data['contract_until']= data['contract_until'].apply(func) 
    
    data.drop(['name','id','position','prefer_foot'], axis=1, inplace= True)

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data = scaler.fit_transform(data)

In [6]:
train.shape, test.shape

((8932, 8), (3828, 7))

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8932 entries, 0 to 8931
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               8932 non-null   int64  
 1   continent         8932 non-null   int64  
 2   contract_until    8932 non-null   int64  
 3   reputation        8932 non-null   float64
 4   stat_overall      8932 non-null   int64  
 5   stat_potential    8932 non-null   int64  
 6   stat_skill_moves  8932 non-null   float64
 7   value             8932 non-null   float64
dtypes: float64(3), int64(5)
memory usage: 558.4 KB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3828 entries, 0 to 3827
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               3828 non-null   int64  
 1   continent         3828 non-null   int64  
 2   contract_until    3828 non-null   int64  
 3   reputation        3828 non-null   float64
 4   stat_overall      3828 non-null   int64  
 5   stat_potential    3828 non-null   int64  
 6   stat_skill_moves  3828 non-null   float64
dtypes: float64(2), int64(5)
memory usage: 209.5 KB


## 2.2. 모델 적용

In [9]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [10]:
inputs = train.drop('value',axis = 1)  # or train.drop(columns = 'Survived')  #문제

In [11]:
outputs = train['value'] #정답지

In [12]:
print(inputs.shape, outputs.shape)  #확인 해야 한다.

(8932, 7) (8932,)


In [13]:
train_input , test_input, train_output, test_output = train_test_split(inputs, outputs, test_size = 0.2 , random_state = 10) 

#test_size train이 8 test가 2로  학습에 더 많은 비중 / stratify ->  output에 있는 데이터의 비율을 유지하면서 값을 뽑아준다 / random_state => seed 보통 10

In [14]:
model1 = Ridge()
model2 = LogisticRegression()
model3 = DecisionTreeClassifier()
model4 = RandomForestClassifier()
model5 = GradientBoostingClassifier()
model6 = AdaBoostClassifier()

### 2.3.1. Model 2 - LogisticRegression

#### 2.3.1.1. Model 2 - LogisticRegression (penalty='l2')

In [15]:
model2.fit(train_input, train_output)

/Users/kmj/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [16]:
model2.score(train_input, train_output) * 100

4.9545136459062284

In [17]:
model2.score(test_input, test_output) * 100

4.98041410184667

In [18]:
test_pre = model2.predict(test_input)

In [19]:
test_output

2851     1600000.0
3925      600000.0
6183      475000.0
246     11000000.0
1090     2700000.0
           ...    
4275      600000.0
8773       90000.0
7990      230000.0
5       44000000.0
4105      975000.0
Name: value, Length: 1787, dtype: float64

In [20]:
test_pre[0]

1000000.0

In [21]:
test_input.iloc[0]

age                   31.0
continent              0.0
contract_until      2020.0
reputation             1.0
stat_overall          70.0
stat_potential        70.0
stat_skill_moves       3.0
Name: 2851, dtype: float64

In [22]:
accuracy_score(test_pre, test_output) * 100

4.98041410184667

#### 2.3.1.2. Model 2 - LogisticRegression (penalty='none')

In [23]:
model2 = LogisticRegression(penalty='none')

In [24]:
model2.fit(train_input, train_output)

/Users/kmj/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(penalty='none')

In [25]:
model2.score(train_input, train_output) * 100

4.96850944716585

In [26]:
model2.score(test_input, test_output) * 100

5.372132064913263

In [27]:
test_pre = model2.predict(test_input)

In [28]:
test_output

2851     1600000.0
3925      600000.0
6183      475000.0
246     11000000.0
1090     2700000.0
           ...    
4275      600000.0
8773       90000.0
7990      230000.0
5       44000000.0
4105      975000.0
Name: value, Length: 1787, dtype: float64

In [29]:
test_pre[0]

1000000.0

In [30]:
test_input.iloc[0]

age                   31.0
continent              0.0
contract_until      2020.0
reputation             1.0
stat_overall          70.0
stat_potential        70.0
stat_skill_moves       3.0
Name: 2851, dtype: float64

In [31]:
accuracy_score(test_pre, test_output) * 100

5.372132064913263

### 2.3.2. Ridge

#### 2.3.2.1. Ridge (fit_intercept=True, normalize=False) - default

In [32]:
model1 = Ridge()

In [91]:
model1.fit(train_input, train_output, sample_weight=100)

/Users/kmj/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


Ridge(fit_intercept=False, normalize=True)

In [83]:
model1.score(train_input, train_output) * 100

64.17835834933797

In [84]:
model1.score(test_input, test_output) * 100

65.18541246680007

In [85]:
test_pre = model1.predict(test_input)

In [87]:
test_pre[0]

1013829.3246033975

In [88]:
test_input.iloc[0]

age                   31.0
continent              0.0
contract_until      2020.0
reputation             1.0
stat_overall          70.0
stat_potential        70.0
stat_skill_moves       3.0
Name: 2851, dtype: float64

In [89]:
test_output

2851     1600000.0
3925      600000.0
6183      475000.0
246     11000000.0
1090     2700000.0
           ...    
4275      600000.0
8773       90000.0
7990      230000.0
5       44000000.0
4105      975000.0
Name: value, Length: 1787, dtype: float64

#### 2.3.2.2. Ridge (fit_intercept=False, normalize=False)

In [43]:
model1 = Ridge(fit_intercept=False)

In [44]:
model1.fit(train_input, train_output, sample_weight=1)

Ridge(fit_intercept=False)

In [45]:
model1.score(train_input, train_output) * 100

64.17835834933797

In [46]:
model1.score(test_input, test_output) * 100

65.18541246680007

In [47]:
test_pre = model1.predict(test_input)

In [48]:
test_output

2851     1600000.0
3925      600000.0
6183      475000.0
246     11000000.0
1090     2700000.0
           ...    
4275      600000.0
8773       90000.0
7990      230000.0
5       44000000.0
4105      975000.0
Name: value, Length: 1787, dtype: float64

In [49]:
test_pre[0]

1013829.3246033975

In [50]:
test_input.iloc[0]

age                   31.0
continent              0.0
contract_until      2020.0
reputation             1.0
stat_overall          70.0
stat_potential        70.0
stat_skill_moves       3.0
Name: 2851, dtype: float64

#### 2.3.2.3. Ridge (fit_intercept=True, normalize=True)

In [52]:
model1 = Ridge(normalize=True)

In [53]:
model1.fit(train_input, train_output, sample_weight=10)

/Users/kmj/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


Ridge(normalize=True)

In [54]:
model1.score(train_input, train_output) * 100

55.65060864129491

In [55]:
model1.score(test_input, test_output) * 100

56.98924106696527

In [56]:
test_pre = model1.predict(test_input)

In [58]:
test_pre[0]

2599660.3344591856

In [59]:
test_input.iloc[0]

age                   31.0
continent              0.0
contract_until      2020.0
reputation             1.0
stat_overall          70.0
stat_potential        70.0
stat_skill_moves       3.0
Name: 2851, dtype: float64

#### 2.3.2.4. Ridge (fit_intercept=False, normalize=True))

In [61]:
model1 = Ridge(fit_intercept= False, normalize=True)

In [62]:
model1.fit(train_input, train_output, sample_weight=100)

/Users/kmj/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


Ridge(fit_intercept=False, normalize=True)

In [63]:
model1.score(train_input, train_output) * 100

64.17837814694245

In [64]:
model1.score(test_input, test_output) * 100

65.18079853925776

In [65]:
test_pre = model1.predict(test_input)

In [67]:
test_pre[0]

1011840.1426369195

In [68]:
test_input.iloc[0]

age                   31.0
continent              0.0
contract_until      2020.0
reputation             1.0
stat_overall          70.0
stat_potential        70.0
stat_skill_moves       3.0
Name: 2851, dtype: float64

In [ ]:
accuracy_score(test_pre, test_output) * 100